## 1) Gerar Historico GIT - CLI no Documentos/Github

In [4]:
import os
import subprocess
import csv
from datetime import datetime

# Configuração - Aponta para a raiz dos projetos GitHub
root_dir = "/home/pedrov12/Documentos/GitHub/"
output_file = "/home/pedrov12/Documentos/GitHub/Pikachu-Flask-Server/batcaverna/batcaverna-project/scripts/historico-projetos.csv"


def get_git_log(repo_path):
    try:
        # Pega os ultimos 10 commits: Hash Abreviado | Data | Autor | Mensagem
        cmd = [
            "git",
            "-C",
            repo_path,
            "log",
            "-n",
            "10",
            "--pretty=format:%h|%ad|%an|%s",
            "--date=short",
        ]
        result = (
            subprocess.check_output(cmd, stderr=subprocess.STDOUT)
            .decode("utf-8")
            .strip()
        )
        if not result:
            return []
        return result.split("\n")
    except subprocess.CalledProcessError:
        return []
    except Exception as e:
        return [f"Erro|{str(e)}||"]


def main():
    print(f"Iniciando varredura em: {os.path.abspath(root_dir)}")

    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        # Cabeçalho do CSV
        writer.writerow(["Projeto", "Hash", "Data", "Autor", "Mensagem"])

        # Listar diretórios
        items = os.listdir(root_dir)
        items.sort()

        projects_found = 0

        for item in items:
            item_path = os.path.join(root_dir, item)

            # Verifica se é diretório e se tem .git
            if os.path.isdir(item_path) and os.path.exists(
                os.path.join(item_path, ".git")
            ):
                projects_found += 1
                commits = get_git_log(item_path)

                if commits:
                    for commit in commits:
                        parts = commit.split("|")
                        if len(parts) >= 4:
                            # Reconstrói a mensagem se ela continha pipes
                            msg = "|".join(parts[3:])
                            writer.writerow([item, parts[0], parts[1], parts[2], msg])
                        else:
                            writer.writerow([item, "Formato Inválido", "", "", commit])
                else:
                    writer.writerow([item, "Sem commits", "", "", ""])

    print(
        f"Concluido. {projects_found} projetos git analisados. Arquivo salvo em {output_file}"
    )


if __name__ == "__main__":
    main()

Iniciando varredura em: /home/pedrov12/Documentos/GitHub
Concluido. 8 projetos git analisados. Arquivo salvo em /home/pedrov12/Documentos/GitHub/Pikachu-Flask-Server/batcaverna/batcaverna-project/scripts/historico-projetos.csv


## 2) Varredura de historico de projetos

In [5]:
import os
import subprocess
import csv
from datetime import datetime

# Configuração - Aponta para a raiz dos projetos GitHub
root_dir = "/home/pedrov12/Documentos/GitHub/"
output_file = "/home/pedrov12/Documentos/GitHub/Pikachu-Flask-Server/batcaverna/batcaverna-project/scripts/historico-projetos.csv"


def get_git_log(repo_path):
    try:
        # Pega os ultimos 10 commits: Hash Abreviado | Data | Autor | Mensagem
        cmd = [
            "git",
            "-C",
            repo_path,
            "log",
            "-n",
            "10",
            "--pretty=format:%h|%ad|%an|%s",
            "--date=short",
        ]
        result = (
            subprocess.check_output(cmd, stderr=subprocess.STDOUT)
            .decode("utf-8")
            .strip()
        )
        if not result:
            return []
        return result.split("\n")
    except subprocess.CalledProcessError:
        return []
    except Exception as e:
        return [f"Erro|{str(e)}||"]


def main():
    print(f"Iniciando varredura em: {os.path.abspath(root_dir)}")

    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        # Cabeçalho do CSV
        writer.writerow(["Projeto", "Hash", "Data", "Autor", "Mensagem"])

        # Listar diretórios
        items = os.listdir(root_dir)
        items.sort()

        projects_found = 0

        for item in items:
            item_path = os.path.join(root_dir, item)

            # Verifica se é diretório e se tem .git
            if os.path.isdir(item_path) and os.path.exists(
                os.path.join(item_path, ".git")
            ):
                projects_found += 1
                commits = get_git_log(item_path)

                if commits:
                    for commit in commits:
                        parts = commit.split("|")
                        if len(parts) >= 4:
                            # Reconstrói a mensagem se ela continha pipes
                            msg = "|".join(parts[3:])
                            writer.writerow([item, parts[0], parts[1], parts[2], msg])
                        else:
                            writer.writerow([item, "Formato Inválido", "", "", commit])
                else:
                    writer.writerow([item, "Sem commits", "", "", ""])

    print(
        f"Concluido. {projects_found} projetos git analisados. Arquivo salvo em {output_file}"
    )


if __name__ == "__main__":
    main()

Iniciando varredura em: /home/pedrov12/Documentos/GitHub
Concluido. 8 projetos git analisados. Arquivo salvo em /home/pedrov12/Documentos/GitHub/Pikachu-Flask-Server/batcaverna/batcaverna-project/scripts/historico-projetos.csv


## 3) Análise de projetos GitHub

In [6]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display


file_path ="/home/pedrov12/Documentos/GitHub/Pikachu-Flask-Server/batcaverna/batcaverna-project/scripts/historico-projetos.csv"


def analisar():
    if not os.path.exists(file_path):
        print(f"Erro: O arquivo {file_path} não foi encontrado.")
        return

    try:
        # 1. Carregar Dados
        df = pd.read_csv(file_path)

        # 2. Converter Data e Ordenar
        df["Data"] = pd.to_datetime(df["Data"], errors="coerce")
        df = df.dropna(subset=["Data"])  # Remove datas inválidas
        df = df.sort_values(by="Data", ascending=True)  # Mais antigo para o mais novo

        print("-" * 60)
        print("🔍 1. AMOSTRA DOS DADOS (ORDENADOS POR DATA)")
        print("-" * 60)
        # Mostra as colunas principais das últimas 10 linhas (mais recentes)
        display(df[["Data", "Projeto", "Mensagem"]].sort_values(by="Data", ascending=False).head(20))

        # 3. Análise Mensal (Agrupamento)
        df["Periodo"] = df["Data"].dt.to_period(
            "M"
        )  # Cria coluna Jan-2025, Fev-2025...

        # Tabela Cruzada: Linhas = Mês, Colunas = Projeto, Valores = Contagem de Commits
        pivot_mensal = pd.crosstab(df["Periodo"], df["Projeto"])

        print("\n" + "-" * 60)
        print("📅 2. HISTÓRICO DE ATIVIDADE MENSAL (QTD COMMITS)")
        print("-" * 60)
        display(pivot_mensal)

        # 4. Total Geral por Mês (Acumulado do mês)
        total_por_mes = df.groupby("Periodo").size()

        print("\n" + "-" * 60)
        print("📈 3. EVOLUÇÃO TOTAL (SOMA DE TODOS OS PROJETOS)")
        print("-" * 60)
        display(total_por_mes)

        return df

    except Exception as e:
        print(f"Ocorreu um erro na análise: {e}")
        return None


def plotar_serie_temporal(df):
    """Plota série temporal com commits por projeto"""
    if df is None or df.empty:
        print("Erro: DataFrame vazio ou inválido.")
        return

    try:
        # Agrupa por data e projeto, contando commits
        df_temporal = df.groupby(["Data", "Projeto"]).size().reset_index(name="Commits")

        # Cria figura com tamanho maior
        plt.figure(figsize=(14, 7))

        # Plota cada projeto como uma linha
        for projeto in df_temporal["Projeto"].unique():
            dados_projeto = df_temporal[df_temporal["Projeto"] == projeto].sort_values(
                "Data"
            )
            plt.plot(
                dados_projeto["Data"],
                dados_projeto["Commits"],
                marker="o",
                label=projeto,
                linewidth=2,
            )

        plt.xlabel("Data", fontsize=12, fontweight="bold")
        plt.ylabel("Quantidade de Commits", fontsize=12, fontweight="bold")
        plt.title("Série Temporal: Commits por Projeto", fontsize=14, fontweight="bold")
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=9)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Salva a figura
        plt.savefig(
            "/home/pedrov12/Documentos/GitHub/serie_temporal_commits.png",
            dpi=300,
            bbox_inches="tight",
        )
        print(
            "\n✅ Gráfico de série temporal salvo em: /home/pedrov12/Documentos/GitHub/serie_temporal_commits.png"
        )

        plt.show()

    except Exception as e:
        print(f"Erro ao plotar série temporal: {e}")


if __name__ == "__main__":
    print("=" * 60)
    print("ANÁLISE DE HISTÓRICO DE PROJETOS GIT")
    print("=" * 60)

    df = analisar()

    print("\n" + "=" * 60)
    print("GERANDO GRÁFICO DE SÉRIE TEMPORAL")
    print("=" * 60)
    plotar_serie_temporal(df)

ANÁLISE DE HISTÓRICO DE PROJETOS GIT


------------------------------------------------------------
🔍 1. AMOSTRA DOS DADOS (ORDENADOS POR DATA)
------------------------------------------------------------


,Data,Projeto,Mensagem
44,2026-02-11,Repopulation-With-Elite-Set,novas metas e upgrades 2026
10,2026-02-11,Jedi-CyberPunk,update PVRV
11,2026-02-11,Jedi-CyberPunk,update estudos e coding no linux
65,2026-02-11,dashboard-website-template,up
64,2026-02-11,dashboard-website-template,Update index.html
34,2026-02-11,Pikachu-Flask-Server,up
36,2026-02-10,Pikachu-Flask-Server,up
13,2026-02-10,Jedi-CyberPunk,Update Terapia_2026_TCC.md
69,2026-02-10,dashboard-website-template,up
68,2026-02-10,dashboard-website-template,up



------------------------------------------------------------
📅 2. HISTÓRICO DE ATIVIDADE MENSAL (QTD COMMITS)
------------------------------------------------------------


Projeto,Gohan-treinamentos-web-app,Jedi-CyberPunk,Network-Tunnel-Go-VM,Palkia-PDF-extractor,Pikachu-Flask-Server,Repopulation-With-Elite-Set,astro-blog-pedrov12,dashboard-website-template
Periodo,,,,,,,,
2025-06,4,0,0,0,0,0,0,0
2025-07,1,0,0,0,0,0,0,0
2025-08,2,0,0,0,0,0,0,0
2025-09,0,0,0,0,0,0,6,0
2025-12,0,0,0,0,0,7,1,0
2026-01,2,3,4,5,0,0,3,2
2026-02,1,7,0,5,10,3,0,8



------------------------------------------------------------
📈 3. EVOLUÇÃO TOTAL (SOMA DE TODOS OS PROJETOS)
------------------------------------------------------------


Periodo
2025-06     4
2025-07     1
2025-08     2
2025-09     6
2025-12     8
2026-01    19
2026-02    34
Freq: M, dtype: int64


GERANDO GRÁFICO DE SÉRIE TEMPORAL

✅ Gráfico de série temporal salvo em: /home/pedrov12/Documentos/GitHub/serie_temporal_commits.png


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

# Ler o arquivo
df = pd.read_excel(
    "/home/pedrov12/Documentos/GitHub/Pikachu-Flask-Server/batcaverna/planilhas/Tarefas_PLC_ONS_10_02_2026.xlsx"
)

# Contar Status
status_counts = df["Concluído"].value_counts()

# Plotar
status_counts.plot(kind="bar", color=["green", "red"])
plt.title("Produtividade Diária")
plt.show()